<a href="https://colab.research.google.com/github/Nov05/wandb-edu/blob/main/mlops-001/lesson1/02_Split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

notebook changed by nov05 on 2023-02-24  
go to [the github file](https://github.com/Nov05/wandb-edu/blob/main/mlops-001/lesson1/02_Split.ipynb), view it at [nbviewer](https://nbviewer.org/github/Nov05/wandb-edu/blob/92ecc403de4132f654e581dd9fac3e44f8bb8359/mlops-001/lesson1/02_Split.ipynb)   
repo `wandb-edu` file path, `mlops-001/lesson1/02_Split.ipynb`  

In [ ]:
!wget https://raw.githubusercontent.com/Nov05/wandb-edu/2393150ad396928ab1421b0e2d6b259863348dcf/mlops-001/lesson1/utils.py
!wget https://raw.githubusercontent.com/Nov05/wandb-edu/2393150ad396928ab1421b0e2d6b259863348dcf/mlops-001/lesson1/params.py

In [ ]:
!pip install wandb
## wandb-0.13.10

<a href="https://colab.research.google.com/github/wandb/edu/blob/main/mlops-001/lesson1/02_Split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{course-lesson1} -->

# Data preparation

<!--- @wandbcode{course-lesson1} -->

In this notebook we will prepare the data to later train our deep learning model. To do so,

- we will start a new W&B `run` and use our raw data artifact
- split the data and save the splits into a new W&B Artifact
- join information about the split with our EDA Table

In [5]:
import os, warnings
import wandb

import pandas as pd
from fastai.vision.all import *
from sklearn.model_selection import StratifiedGroupKFold

import params
warnings.filterwarnings('ignore')

In [6]:
run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="data_split")

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Let's use artifact we previously saved to W&B (we're storing artifact names and other global parameters in `params`).

In [21]:
raw_data_at = run.use_artifact(f'{params.RAW_DATA_AT}:latest')
path = Path(raw_data_at.download())

wandb: Downloading large artifact bdd_simple_1k:latest, 813.75MB. 4007 files... 
wandb:   4007 of 4007 files downloaded.  
Done. 0:0:39.0


In [22]:
path.ls()

(#5) [Path('artifacts/bdd_simple_1k:v1/LICENSE.txt'),Path('artifacts/bdd_simple_1k:v1/media'),Path('artifacts/bdd_simple_1k:v1/eda_table.table.json'),Path('artifacts/bdd_simple_1k:v1/images'),Path('artifacts/bdd_simple_1k:v1/labels')]

To split data between training, testing and validation, we need file names, groups (derived from the file name) and target (here we use our rare class bicycle for stratification). We previously saved these columns to EDA table, so let's retrieve it from the table now. 

In [24]:
fnames = os.listdir(path/'images')
groups = [s.split('-')[0] for s in fnames]
print(fnames[0], len(fnames))
print(groups[0], len(groups))

373e20c8-82cb9d82.jpg 1000
373e20c8 1000


In [25]:
orig_eda_table = raw_data_at.get("eda_table")

wandb: Downloading large artifact bdd_simple_1k:latest, 813.75MB. 4007 files... 
wandb:   4007 of 4007 files downloaded.  
Done. 0:0:0.6


In [26]:
y = orig_eda_table.get_column('bicycle')
print(len(y))

1000


Now we will split the data into train (80%), validation (10%) and test (10%) sets. As we do that, we need to be careful to:

- *avoid leakage*: for that reason we are grouping data according to video identifier (we want to make sure our model can generalize to new cars or video frames)
- handle the *label imbalance*: for that reason we stratify data with our target column

We will use sklearn's `StratifiedGroupKFold` to split the data into 10 folds and assign 1 fold for test, 1 for validation and the rest for training.

In [27]:
df = pd.DataFrame()
df['File_Name'] = fnames
df['fold'] = -1

In [28]:
cv = StratifiedGroupKFold(n_splits=10)
for i, (train_idxs, test_idxs) in enumerate(cv.split(fnames, y, groups)):
    df.loc[test_idxs, ['fold']] = i
## if you run into the following error, go back to 01_EDA.ipynb and set "DEBUG = False"
## ValueError: Found input variables with inconsistent numbers of samples: [1000, 10, 1000]

In [29]:
df['Stage'] = 'train'
df.loc[df.fold == 0, ['Stage']] = 'test'
df.loc[df.fold == 1, ['Stage']] = 'valid'
del df['fold']
df.Stage.value_counts()

train    800
test     100
valid    100
Name: Stage, dtype: int64

In [30]:
df.to_csv('data_split.csv', index=False)

We will now create a new artifact and add our data there. 

In [31]:
processed_data_at = wandb.Artifact(params.PROCESSED_DATA_AT, type="split_data")

In [32]:
processed_data_at.add_file('data_split.csv')
processed_data_at.add_dir(path)

wandb: Adding directory to artifact (./artifacts/bdd_simple_1k:v1)... Done. 7.5s


Finally, the split information may be relevant for our analyses - rather than uploading images again, we will save the split information to a new table and join it with EDA table we created previously. 

In [33]:
data_split_table = wandb.Table(dataframe=df[['File_Name', 'Stage']])

In [34]:
join_table = wandb.JoinedTable(orig_eda_table, data_split_table, "File_Name")

Let's add it to our artifact, log it and finish our `run`. 

In [35]:
processed_data_at.add(join_table, "eda_table_data_split")

ArtifactManifestEntry(path='eda_table_data_split.joined-table.json', digest='rBpUlYKsLTATlaDODsUNdg==', ref=None, birth_artifact_id=None, size=127, extra={}, local_path='/root/.local/share/wandb/artifacts/staging/tmpc3_cxnb4')

In [36]:
run.log_artifact(processed_data_at)
run.finish()